# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
# sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/temp.csv')

## The H&M Dataset

In [3]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499999 entries, 0 to 499998
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             499999 non-null  object 
 1   customer_id       499999 non-null  object 
 2   article_id        499999 non-null  int64  
 3   price             499999 non-null  float64
 4   sales_channel_id  499999 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 19.1+ MB


In [6]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [7]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

In [8]:
articles_sample = pd.read_csv('../data/articles_sample01.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample01.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample01.csv.gz')

In [9]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             1371 non-null   object 
 1   FN                      514 non-null    float64
 2   Active                  504 non-null    float64
 3   club_member_status      1366 non-null   object 
 4   fashion_news_frequency  1360 non-null   object 
 5   age                     1358 non-null   float64
 6   postal_code             1371 non-null   object 
dtypes: float64(3), object(4)
memory usage: 75.1+ KB


In [10]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   t_dat             513 non-null    object 
 1   customer_id       513 non-null    object 
 2   article_id        513 non-null    int64  
 3   price             513 non-null    float64
 4   sales_channel_id  513 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 20.2+ KB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [11]:
# If you'd rather use a sample. Uncomment the following code:
# transactions = transactions_sample
# customers = customers_sample
# articles = articles_sample

In [12]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,1
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,1


In [14]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [15]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,1
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,1
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,1


In [16]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [17]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

499999


In [18]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [19]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [20]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-23,3207a8a93ab3a978cd4ad4ff719b9b644cec95cc18840c...,714418001,0.0338814,2,0
1,2018-09-28,7bda69f018a1b7be2bacdf712d8aaf7a1bacf052b4b572...,176550020,0.0169322,1,0
2,2018-09-25,04e6fea09c850352c46cc54afbe0b62db1e497eab630fa...,515158001,0.0254068,1,0
3,2018-09-24,2427a1e6c33d3c9c47c53182d39ba9dbb68e59f9a146d4...,624661002,0.0135424,2,0
4,2018-09-24,32db696f74299dd12ebfd257dd76f06d8b2b6be8dede3f...,620855001,0.0152373,1,0


In [21]:
neg_transactions.shape

(549998, 6)

In [22]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [23]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [24]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999998 entries, 0 to 999997
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   t_dat                         999998 non-null  object 
 1   customer_id                   999998 non-null  object 
 2   article_id                    999998 non-null  object 
 3   price                         999998 non-null  object 
 4   sales_channel_id              999998 non-null  object 
 5   ordered                       999998 non-null  object 
 6   FN                            410991 non-null  float64
 7   Active                        401514 non-null  float64
 8   club_member_status            988928 non-null  object 
 9   fashion_news_frequency        995383 non-null  object 
 10  age                           995291 non-null  float64
 11  postal_code                   999998 non-null  object 
 12  product_code                  999998 non-nul

### Basic Preprocessing
Some very basic preprocessing.

In [25]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,2,0.0508305,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,2,0.0508305,1
2,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,30.0,663713001,2,0.0494746,1
3,429302530c2e74d1d230e836023b93071ddbf3f5ad1107...,57.0,663713001,2,0.0508305,0
4,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,37.0,663713001,2,0.0508305,1


In [26]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [27]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [28]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2'],
      dtype=object)

In [29]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,1,24.0,21488,2,0.0508305,1
1,1,24.0,21488,2,0.0508305,1
2,28068,30.0,21488,2,0.0494746,1
3,34303,57.0,21488,2,0.0508305,0
4,40832,37.0,21488,2,0.0508305,1


In [30]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [31]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [32]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,1,24.0,21488,0.050831,1,0,1
1,1,24.0,21488,0.050831,1,0,1
2,28068,30.0,21488,0.049475,1,0,1
3,34303,57.0,21488,0.050831,0,0,1
4,40832,37.0,21488,0.050831,1,0,1


In [33]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [34]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
378046,21895,24.0,11175,0.033881,0,1
510062,87354,31.0,20397,0.010153,0,1
534815,119250,26.0,11308,0.008458,0,1
342590,38958,34.0,13653,0.064390,1,0
559414,89894,26.0,8121,0.011847,1,0


In [35]:
y_train.head()

378046    1
510062    1
534815    1
342590    0
559414    1
Name: ordered, dtype: int64

## Baseline Model

In [36]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [37]:
baseline.predict_proba(X_test)

array([[0.5384392 , 0.4615608 ],
       [0.48492453, 0.51507547],
       [0.50096165, 0.49903835],
       ...,
       [0.47828686, 0.52171314],
       [0.48197963, 0.51802037],
       [0.48827761, 0.51172239]])

In [38]:
y_test

987229    0
79954     1
567130    0
500891    1
55399     0
         ..
395942    1
417771    1
713259    0
794020    0
573083    0
Name: ordered, Length: 100000, dtype: int64

In [39]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.51373

In [40]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.43      0.47     49861
           1       0.51      0.60      0.55     50139

    accuracy                           0.51    100000
   macro avg       0.51      0.51      0.51    100000
weighted avg       0.51      0.51      0.51    100000



In [41]:
predictions

array([0, 1, 0, ..., 1, 1, 1])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


In [42]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'price', 'postal_code', 't_dat', 'colour_group_code', 'ordered']].copy()

In [43]:
# Impute the nan values in the age column with median value
median_age = transactions_processed['age'].median()
transactions_processed['age'] = transactions_processed['age'].fillna(median_age)
# Imputing the rest of the NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)

In [44]:
# FEATURE: calculate diff with latest bought
transactions_processed['t_dat'] = pd.to_datetime(transactions_processed['t_dat'], format='%Y-%m-%d')
transactions_processed['last_bought'] = transactions_processed['t_dat'].max()
transactions_processed['diff'] = (transactions_processed['last_bought'] - transactions_processed['t_dat']).dt.days
transactions_processed = transactions_processed.drop(['last_bought', 't_dat'], axis=1)
# FEATURE: count popularity per age bracket
transactions_processed['age_bracket'] = transactions_processed['age'] // 10
article_counts = transactions_processed[['article_id', 'colour_group_code']].value_counts()
article_counts.name = 'count_age'
transactions_processed = transactions_processed.merge(article_counts, on=['article_id', 'colour_group_code'])
# FEATURE: count transactions per postal code group
article_counts = transactions_processed[['article_id', 'postal_code']].value_counts()
article_counts.name = 'count_postal_code'
transactions_processed = transactions_processed.merge(article_counts, on=['article_id', 'postal_code'])
# FEATURE: count transactions per colour group
article_counts = transactions_processed[['article_id', 'colour_group_code']].value_counts()
article_counts.name = 'count_colour'
transactions_processed = transactions_processed.merge(article_counts, on=['article_id', 'colour_group_code'])
transactions_processed

,customer_id,age,article_id,price,postal_code,colour_group_code,ordered,diff,age_bracket,count_age,count_postal_code,count_colour
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,0.050831,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,9,1,9,2.0,101,2,101
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,0.050831,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,9,1,5,2.0,101,2,101
2,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,30.0,663713001,0.049475,aea9af1fd9b5d3aa98820e66a7a093f54def0e4db44153...,9,1,9,3.0,101,1,101
3,429302530c2e74d1d230e836023b93071ddbf3f5ad1107...,57.0,663713001,0.050831,0f8ff57c1f243f5c2c7aae7b3f21a767b4cf419e118bff...,9,0,7,5.0,101,1,101
4,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,37.0,663713001,0.050831,7adb1bc2fa962cfb71b31c97223a5b03bb7886e2be56c9...,9,1,9,3.0,101,1,101
...,...,...,...,...,...,...,...,...,...,...,...,...
999993,a2db2373943e322f46aa34d177c9e3b7a4ff64a133e52f...,68.0,553518007,0.015237,c36047c259cdf40bf22fbe34c609533bee323338782261...,12,0,7,6.0,8,1,8
999994,2c443284739efecc16e07cef758679868ba8d4ac3875de...,67.0,553518007,0.015237,84eefbc081c719cb7f45bbf7a1cb3a40533b146e85c4e0...,12,0,9,6.0,8,1,8
999995,43994d39beb6ec372f599f9df2386ed03a857498f73881...,24.0,553518007,0.015237,b360f7323fa693951fe2cdb2f7b2c88bbe99923a4545e6...,12,0,1,2.0,8,1,8
999996,2c329773998ed69c0c6c4f1fa092b937c2c9f972389fb3...,26.0,553518007,0.015237,6362198700033f9feb256c5d20b988484bd8cbe7409f8a...,12,0,8,2.0,8,1,8


In [45]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
postal_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed['postal_code'] = postal_encoder.fit_transform(transactions_processed['postal_code'])

In [46]:
transactions_processed = transactions_processed.replace([np.inf, -np.inf], 0)
scaler = preprocessing.MinMaxScaler()
transactions_processed[['price']] = scaler.fit_transform(transactions_processed[['price']])
transactions_processed[['price']]

,price
0,0.084882
1,0.084882
2,0.082587
3,0.084882
4,0.084882
...,...
999993,0.024641
999994,0.024641
999995,0.024641
999996,0.024641


In [47]:
transactions_processed = transactions_processed.merge(transactions_processed.groupby('postal_code')[['price']].median(), how='inner', on='postal_code')
transactions_processed['price_y']

0         0.056211
1         0.056211
2         0.056211
3         0.056211
4         0.056211
            ...   
999993    0.056196
999994    0.170941
999995    0.005709
999996    0.017183
999997    0.099225
Name: price_y, Length: 999998, dtype: float64

Cleaning up the nan values and custom processing the price column

In [48]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
print(baseline.score(X_test, y_test))
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

/home/basil/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.75548
              precision    recall  f1-score   support

           0       0.70      0.88      0.78     49960
           1       0.84      0.63      0.72     50040

    accuracy                           0.76    100000
   macro avg       0.77      0.76      0.75    100000
weighted avg       0.77      0.76      0.75    100000

